In [ ]:
!pip install apache_beam[interactive] stopwords --q

import numpy

import tensorflow as tf

import tensorflow_datasets as tfds

tfds.disable_progress_bar()

In [5]:
import tensorflow.keras.layers as layers

import tensorflow.keras.utils as utils

data = "imdb_reviews"; ontrue = True

(train, testi), info = tfds.load(data, split=["train", "test"], with_info=ontrue, as_supervised=ontrue)

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
info.splits["train"].num_examples, info.splits["test"].num_examples

(25000, 25000)

In [7]:
from stopwords import get_stopwords

unlist = get_stopwords("english")

In [8]:
train_input = []
train_label = []

testi_input = []
testi_label = []

retori = lambda snt : [i for i in snt.lower().split() if i not in unlist]

for data, label in train:
  retari = " ".join(retori(data.numpy().decode("utf8")))
  train_input.append(retari)
  train_label.append(label.numpy())

for data, label in testi:
  retari = " ".join(retori(data.numpy().decode("utf8")))
  testi_input.append(retari)
  testi_label.append(label.numpy())

In [9]:
len(train_input), len(testi_input), len(train_label), len(testi_label)

(25000, 25000, 25000, 25000)

In [10]:
vector = layers.TextVectorization(max_tokens=10)

sample = ["aku suka ikan", "ikan goreng", "ibu masak ikan"]

vector.adapt(sample)

In [11]:
vector.get_vocabulary()

['', '[UNK]', 'ikan', 'suka', 'masak', 'ibu', 'goreng', 'aku']

In [12]:
vector.get_vocabulary()[5]

'ibu'

In [13]:
samplet = ["ikan masak ibu tadi"]

vector(samplet).numpy()

array([[2, 4, 5, 1]])

In [14]:
len(vector.get_vocabulary())

8

In [20]:
# del vector

max_tokens = 15000

embedding = 16

output_sequence_length = 180

tokener = layers.TextVectorization(
  max_tokens=max_tokens, output_mode="int",
  pad_to_max_tokens=ontrue,
  output_sequence_length=output_sequence_length)

tokener.adapt(train_input)

In [21]:
train_input = numpy.array(tokener(train_input))

testi_input = numpy.array(tokener(testi_input))

In [22]:
train_label = numpy.array(train_label)

train_label[:5]

array([0, 0, 0, 1, 1])

In [23]:
testi_label = numpy.array(testi_label)

testi_label[:5]

array([1, 1, 0, 0, 1])

In [24]:
inputs = layers.Input(shape=(output_sequence_length,))

x = layers.Embedding(max_tokens, embedding)(inputs)
x = layers.Dropout(0.2)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [25]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

plot = model.fit(train_input, train_label, epochs=5, validation_data=(testi_input, testi_label))

Epoch 1/5
782/782 [==============================] - 67s 82ms/step - loss: 0.6462 - accuracy: 0.7528 - val_loss: 0.5723 - val_accuracy: 0.8275
Epoch 2/5
782/782 [==============================] - 21s 27ms/step - loss: 0.4810 - accuracy: 0.8510 - val_loss: 0.4396 - val_accuracy: 0.8490
Epoch 3/5
782/782 [==============================] - 13s 17ms/step - loss: 0.3687 - accuracy: 0.8785 - val_loss: 0.3685 - val_accuracy: 0.8670
Epoch 4/5
782/782 [==============================] - 12s 15ms/step - loss: 0.3076 - accuracy: 0.8958 - val_loss: 0.3364 - val_accuracy: 0.8708
Epoch 5/5
782/782 [==============================] - 8s 10ms/step - loss: 0.2688 - accuracy: 0.9077 - val_loss: 0.3155 - val_accuracy: 0.8753


In [26]:
model.evaluate(train_input, train_label)

782/782 [==============================] - 2s 3ms/step - loss: 0.2448 - accuracy: 0.9164


[0.2447609007358551, 0.9163600206375122]

In [27]:
model.evaluate(testi_input, testi_label)

782/782 [==============================] - 2s 2ms/step - loss: 0.3155 - accuracy: 0.8753


[0.31552815437316895, 0.8752800226211548]

In [28]:
import os

model.save("/content/modeler")

In [29]:
import warnings

import apache_beam as beam

warnings.filterwarnings("ignore")

In [30]:
from apache_beam.ml.inference.tensorflow_inference import TFModelHandlerTensor

tensor = TFModelHandlerTensor("/content/modeler")

from apache_beam.ml.inference.base import RunInference

In [31]:
trainer = train_input[:10]

len(trainer)

10

In [32]:
falter = lambda t : t.inference.numpy()[0].round()

In [33]:
with beam.Pipeline() as pipeline:
  runner = (
    pipeline
    | 'Create' >> beam.Create(trainer)
    | 'RunInference' >> RunInference(tensor)
    | 'Outer' >> beam.Map(falter)
    | 'Unlist' >> beam.combiners.ToList()
    | 'Output' >> beam.Map(print)
  )

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]


In [ ]:
train_label[:10]

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])

In [ ]:
errori = tf.keras.losses.BinaryCrossentropy()

class ValError(beam.DoFn):
  def process(self, element):
    yield errori(train_label[:10], element).numpy()

In [ ]:
with beam.Pipeline() as pipeline:
  runner = (
    pipeline
    | 'Create' >> beam.Create(trainer)
    | 'RunInference' >> RunInference(tensor)
    | 'Outer' >> beam.Map(falter)
    | 'Unlist' >> beam.combiners.ToList()
    | 'Unruly' >> beam.ParDo(ValError())
    | 'Output' >> beam.LogElements()
  )

3.0666478
